In [1]:
# Import necessary libraries
import os
import json
import time
import pandas as pd
from IPython.display import display
from system import System
from analysis import Analysis

# Define the path to your test cases
# Change this path to match your actual test cases location
root_folder = "DRTS_Project-Test-Cases-master"

def analyze_all_direct(root_folder):
    #"""Directly analyze all test cases using AnalysisTool (no subprocess)"""
    # Get all test case folders
    all_folders = [d for d in os.listdir(root_folder) 
                  if os.path.isdir(os.path.join(root_folder, d)) and d[0].isdigit()]
    
    # Sort by number
    all_folders.sort(key=lambda x: int(x.split('-')[0]))
    
    print(f"Found {len(all_folders)} test cases:")
    for folder in all_folders:
        print(f"  - {folder}")
    
    results = []
    
    # Analyze each test case
    for folder in all_folders:
        test_path = os.path.join(root_folder, folder)
        
        print(f"\n{'='*60}")
        print(f"Test case: {folder}")
        print(f"{'='*60}")
        
        # Build file paths
        tasks_file = os.path.join(test_path, "tasks.csv")
        arch_file = os.path.join(test_path, "architecture.csv")
        budgets_file = os.path.join(test_path, "budgets.csv")
        
        # Check if files exist
        if not all(os.path.exists(f) for f in [tasks_file, arch_file, budgets_file]):
            print(f"Error: Test case {folder} is missing required input files")
            continue
        
        start_time = time.time()
        
        try:
            # Load system
            print(f"Loading system configuration...")
            system = System(tasks_file, arch_file, budgets_file)
            
            # Create analysis tool
            print("Initializing analysis tool...")
            analyzer = Analysis(system)
            
            # Perform analysis
            print("Running analysis...")
            analyzer.analyze()
            
            # Generate report
            print("Generating analysis report...")
            report = analyzer.generate_report()
            
            # Check if test case is expected to be unschedulable
            expected_unschedulable = "unschedulable" in folder.lower()
            actual_schedulable = report.get("system_schedulable", False)
            
            if expected_unschedulable and actual_schedulable:
                print("Warning: Test case expected to be unschedulable, but analysis shows it's schedulable")
            elif not expected_unschedulable and not actual_schedulable:
                print("Warning: Test case expected to be schedulable, but analysis shows it's not schedulable")
            
            # Save results
            results_file = os.path.join(test_path, "analysis_results.json")
            with open(results_file, 'w') as f:
                json.dump(report, f, indent=4)
            
            # Create CSV output
            solution_file = os.path.join(test_path, "solution.csv")
            create_solution_csv(report, solution_file)
            
            # Get component and task counts
            num_components = len(report.get('components', {}))
            task_count = 0
            for comp in report.get('components', {}).values():
                task_count += len(comp.get('tasks', {}))
            
            # Collect result summary
            results.append({
                "test_case": folder,
                "schedulable": actual_schedulable,
                "expected_unschedulable": expected_unschedulable,
                "match": (expected_unschedulable and not actual_schedulable) or 
                         (not expected_unschedulable and actual_schedulable),
                "components": num_components,
                "tasks": task_count,
                "time": time.time() - start_time
            })
            
            print(f"Results saved to {results_file}")
            
            # Print analysis summary for this test case
            print("\nAnalysis Summary:")
            print(f"System schedulable: {'Yes' if actual_schedulable else 'No'}")
            print(f"Expected result: {'Unschedulable' if expected_unschedulable else 'Schedulable'}")
            print(f"Analysis time: {results[-1]['time']:.2f} seconds")
            
            # Print component details
            print("\nComponent Details:")
            for comp_id, comp_info in report.get('components', {}).items():
                print(f"  Component {comp_id}: {'Schedulable' if comp_info.get('schedulable', False) else 'Not schedulable'}")
                if 'bdr_alpha' in comp_info and 'bdr_delta' in comp_info:
                    print(f"    BDR parameters: alpha={comp_info['bdr_alpha']:.3f}, delta={comp_info['bdr_delta']:.3f}")
            
        except Exception as e:
            print(f"Error analyzing {folder}: {str(e)}")
            import traceback
            traceback.print_exc()
            
            results.append({
                "test_case": folder,
                "schedulable": "Error",
                "expected_unschedulable": expected_unschedulable,
                "match": False,
                "components": "Unknown",
                "tasks": "Unknown",
                "time": time.time() - start_time,
                "error": str(e)
            })
    
    # Create and display summary table
    print("\n" + "="*100)
    print("ANALYSIS RESULTS SUMMARY")
    print("="*100)
    
    headers = ["Test Case", "Expected", "Result", "Match", "Components", "Tasks", "Time (s)"]
    rows = []
    
    for r in results:
        expected = "Unschedulable" if r["expected_unschedulable"] else "Schedulable"
        result = r["schedulable"] if isinstance(r["schedulable"], str) else ("Schedulable" if r["schedulable"] else "Unschedulable")
        match = "✓" if r["match"] else "✗"
        components = r["components"]
        tasks = r["tasks"]
        time_taken = f"{r['time']:.2f}"
        
        rows.append([r['test_case'], expected, result, match, components, tasks, time_taken])
    
    # Create and display a DataFrame for better formatting
    df = pd.DataFrame(rows, columns=headers)
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', 1000)
    display(df)
    
    # Save overall summary
    summary_file = os.path.join(root_folder, "analysis_summary.json")
    with open(summary_file, 'w') as f:
        json.dump(results, f, indent=4)
    
    print(f"\nSummary results saved to: {summary_file}")
    
    # Calculate accuracy
    valid_results = [r for r in results if not isinstance(r["schedulable"], str)]
    if valid_results:
        correct_count = sum(1 for r in valid_results if r["match"])
        accuracy = correct_count / len(valid_results) * 100
        print(f"\nAnalysis accuracy: {correct_count}/{len(valid_results)} ({accuracy:.1f}%)")
        
        # Identify any potential issues
        if accuracy < 100:
            print("\nMismatched test cases:")
            for r in valid_results:
                if not r["match"]:
                    expected = "Unschedulable" if r["expected_unschedulable"] else "Schedulable"
                    actual = "Schedulable" if r["schedulable"] else "Unschedulable"
                    print(f"  - {r['test_case']}: Expected {expected}, got {actual}")
    
    return results

def create_solution_csv(report, output_file):
    """Create solution.csv format output"""
    with open(output_file, 'w') as f:
        # Write header
        f.write("task_name,component_id,task_schedulable,avg_response_time,max_response_time,component_schedulable\n")
        
        # For each component and task in the report
        for comp_id, comp_info in report.get('components', {}).items():
            comp_schedulable = 1 if comp_info.get('schedulable', False) else 0
            
            for task_name, task_info in comp_info.get('tasks', {}).items():
                task_schedulable = 1 if task_info.get('schedulable', True) else 0
                
                # Analysis tool may not have response time info, so use 0 as placeholder
                avg_response_time = task_info.get('avg_response_time', 0)
                max_response_time = task_info.get('max_response_time', 0)
                
                f.write(f"{task_name},{comp_id},{task_schedulable},{avg_response_time},{max_response_time},{comp_schedulable}\n")

# Run the analysis function
# (Uncomment the line below to execute when ready)
results = analyze_all_direct(root_folder)

Found 10 test cases:
  - 1-tiny-test-case
  - 2-small-test-case
  - 3-medium-test-case
  - 4-large-test-case
  - 5-huge-test-case
  - 6-gigantic-test-case
  - 7-unschedulable-test-case
  - 8-unschedulable-test-case
  - 9-unschedulable-test-case
  - 10-unschedulable-test-case

Test case: 1-tiny-test-case
Loading system configuration...
loaded dataframes: 
  core_id  speed_factor scheduler
0  Core_1          0.62        RM
    component_id scheduler  budget  period core_id  priority
0  Camera_Sensor        RM      84      84  Core_1         0
  task_name  wcet  period   component_id  priority
0    Task_0    14      50  Camera_Sensor         0
1    Task_1    33     100  Camera_Sensor         1
Initializing analysis tool...
Running analysis...
Generating analysis report...
Results saved to DRTS_Project-Test-Cases-master\1-tiny-test-case\analysis_results.json

Analysis Summary:
System schedulable: Yes
Expected result: Schedulable
Analysis time: 0.01 seconds

Component Details:
  Component C

,Test Case,Expected,Result,Match,Components,Tasks,Time (s)
0,1-tiny-test-case,Schedulable,Schedulable,✓,1,2,0.01
1,2-small-test-case,Schedulable,Schedulable,✓,2,9,0.01
2,3-medium-test-case,Schedulable,Schedulable,✓,4,18,0.01
3,4-large-test-case,Schedulable,Schedulable,✓,7,28,0.01
4,5-huge-test-case,Schedulable,Schedulable,✓,18,61,0.01
5,6-gigantic-test-case,Schedulable,Schedulable,✓,34,115,0.02
6,7-unschedulable-test-case,Unschedulable,Unschedulable,✓,6,21,0.01
7,8-unschedulable-test-case,Unschedulable,Schedulable,✗,7,28,0.01
8,9-unschedulable-test-case,Unschedulable,Schedulable,✗,18,61,0.01
9,10-unschedulable-test-case,Unschedulable,Schedulable,✗,34,115,0.02



Summary results saved to: DRTS_Project-Test-Cases-master\analysis_summary.json

Analysis accuracy: 7/10 (70.0%)

Mismatched test cases:
  - 8-unschedulable-test-case: Expected Unschedulable, got Schedulable
  - 9-unschedulable-test-case: Expected Unschedulable, got Schedulable
  - 10-unschedulable-test-case: Expected Unschedulable, got Schedulable
